In [1]:
!pip install agentpy

     |████████████████████████████████| 52 kB 1.5 MB/s eta 0:00:011
     |████████████████████████████████| 756 kB 8.6 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 106 kB 53.6 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 5.4 MB/s  eta 0:00:01
     |████████████████████████████████| 86 kB 8.5 MB/s  eta 0:00:01
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.8.1
    Uninstalling importlib-metadata-4.8.1:
      Successfully uninstalled importlib-metadata-4.8.1


In [2]:
!pip install matplotlib==3.4.3

ERROR: Could not find a version that satisfies the requirement matplotlib==3.4.3 (from versions: 0.86, 0.86.1, 0.86.2, 0.91.0, 0.91.1, 1.0.1, 1.1.0, 1.1.1, 1.2.0, 1.2.1, 1.3.0, 1.3.1, 1.4.0, 1.4.1rc1, 1.4.1, 1.4.2, 1.4.3, 1.5.0, 1.5.1, 1.5.2, 1.5.3, 2.0.0b1, 2.0.0b2, 2.0.0b3, 2.0.0b4, 2.0.0rc1, 2.0.0rc2, 2.0.0, 2.0.1, 2.0.2, 2.1.0rc1, 2.1.0, 2.1.1, 2.1.2, 2.2.0rc1, 2.2.0, 2.2.2, 2.2.3, 2.2.4, 2.2.5, 3.0.0rc2, 3.0.0, 3.0.1, 3.0.2, 3.0.3, 3.1.0rc1, 3.1.0rc2, 3.1.0, 3.1.1, 3.1.2, 3.1.3, 3.2.0rc1, 3.2.0rc3, 3.2.0, 3.2.1, 3.2.2, 3.3.0rc1, 3.3.0, 3.3.1, 3.3.2, 3.3.3, 3.3.4)
ERROR: No matching distribution found for matplotlib==3.4.3


In [3]:
!pip install seaborn

     |████████████████████████████████| 292 kB 4.7 MB/s eta 0:00:01


Imports

In [1]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

#Random
import random

Model Definition

In [2]:
class ForestModel(ap.Model):
    
    def setup(self):
        
        # Create agents (trees) 
        n_trees = int(self.p['Tree density'] * (self.p.size**2))
        trees = self.agents = ap.AgentList(self, n_trees)
        
        # Create grid (forest)
        self.forest = ap.Grid(self, [self.p.size]*2, track_empty=True)      
        self.forest.add_agents(trees, random=True, empty=True)
        
        # Initiate a dynamic variable for all trees
        # Condition 0: Alive, 1: Burning, 2: Burned
        self.agents.condition = 0

        self.agents.types = random.randint(0,2)
        
        # Start a fire
        for i in range(self.p['burning Trees']):
            unfortunate_trees = self.forest.agents[random.randint(0,self.p.size-1), random.randint(0,self.p.size-1)]        
            unfortunate_trees.condition = 1
        
    def step(self):
        
        # Select burning trees
        burning_trees = self.agents.select(self.agents.condition == 1)

        # Spread fire 
        for tree in burning_trees:
            for neighbor in self.forest.neighbors(tree):
                if neighbor.types == 0 and neighbor.condition == 0: 
                    if random.randint(0,10) <= 8: #80% chance of catching fire
                      neighbor.condition = 1 # Neighbor starts burning
                if neighbor.types == 1 and neighbor.condition == 0:
                    if random.randint(0,10) <= 5: #50% chances of catching fire
                      neighbor.condition = 1
                if neighbor.types == 2 and neighbor.condition == 0:
                    if random.randint(0,10) <= 2: #20% chances of catching fire
                      neighbor.condition = 1
            tree.condition = 2 # Tree burns out  
        
        # Stop simulation if no fire is left
        if len(burning_trees) == 0: 
            self.stop()
            
    def end(self):
        
        # Document a measure at the end of the simulation
        burned_trees = len(self.agents.select(self.agents.condition == 2))
        self.report('Percentage of burned trees', 
                    burned_trees / len(self.agents))

Run Single Animation

In [3]:
# Define parameters

parameters = {
    'Tree density': 0.6, # Percentage of grid covered by trees
    'size': 50, # Height and length of the grid
    'steps': 100,
    'burning Trees': 100
}

In [4]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.forest.attr_grid('condition')
    color_dict = {0:'#7FC97F', 1:'#d62c2c' , 2:'#e5e5e5', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a forest fire\n"
                 f"Time-step: {model.t}, Trees left: "
                 f"{len(model.agents.select(model.agents.condition == 0))}")

fig, ax = plt.subplots() 
model = ForestModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))

Parameter Sweep

In [6]:
# Prepare parameter sample
parameters = {
    'Tree density': ap.Range(0.2, 0.6), 
    'size': 100
}
sample = ap.Sample(parameters, n=30)

In [ ]:

# Perform experiment
exp = ap.Experiment(ForestModel, sample, iterations=40)
results = exp.run()

Scheduled runs: 1200
Completed: 280, estimated time remaining: 0:01:54

In [ ]:
# Save and load data
results.save()
results = ap.DataDict.load('ForestModel')